In [6]:
!pip install datasets

   ---------------------------------------- 0.0/542.0 kB ? eta -:--:--
   -- ------------------------------------ 30.7/542.0 kB 640.0 kB/s eta 0:00:01
   -- ------------------------------------ 30.7/542.0 kB 640.0 kB/s eta 0:00:01
   ---- ---------------------------------- 61.4/542.0 kB 544.7 kB/s eta 0:00:01
   ------- ------------------------------ 102.4/542.0 kB 737.3 kB/s eta 0:00:01
   ------------ ------------------------- 184.3/542.0 kB 794.9 kB/s eta 0:00:01
   ---------------- --------------------- 235.5/542.0 kB 901.1 kB/s eta 0:00:01
   --------------------- ---------------- 307.2/542.0 kB 948.8 kB/s eta 0:00:01
   ----------------------- -------------- 337.9/542.0 kB 999.0 kB/s eta 0:00:01
   --------------------------- ---------- 399.4/542.0 kB 995.2 kB/s eta 0:00:01
   ----------------------------------- ---- 481.3/542.0 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 542.0/542.0 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
from nltk.util import ngrams
import nltk
import pandas as pd
from nltk.corpus import gutenberg
from nltk.tokenize import sent_tokenize, word_tokenize
import numpy as np
import textstat
from rouge_score import rouge_scorer
from datasets import load_metric
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
import nltk


In [2]:
df = pd.read_csv('cooked_dataset.csv', index_col='Unnamed: 0').reset_index(drop=True)
df.sample()

,title,generated,model,text,clean_text,avg_sentence_len,avg_words_len,char_count,word_count,sentence_count
13950,В США выбрали лучших молодых дизайнеров,0,Human,В Нью-Йорке прошла ежегодная церемония вручени...,в ньюйорке прошла ежегодная церемония вручения...,114.375,5.225166,899,131,8


In [3]:
df['char_count'] = df['text'].apply(len)
df['word_count'] = df['text'].apply(lambda x: len(x.split()))
df['sentence_count'] = df['text'].apply(lambda x: x.count('.') + x.count('!') + x.count('?'))

In [4]:
df

,title,generated,model,text,clean_text,avg_sentence_len,avg_words_len,char_count,word_count,sentence_count
0,Канье Уэст получил пожизненную подписку на Por...,0,Human,Популярный исполнитель Канье Уэст рассказал в ...,популярный исполнитель канье уэст рассказал в ...,87.384615,5.253968,1149,156,14
1,Сторонники Саакашвили сломали дверь микроавтоб...,0,Human,Сторонники Михаила Саакашвили взломали дверь м...,сторонники михаила саакашвили взломали дверь м...,103.000000,5.430000,1248,162,12
2,Мэр Николаева обвинил в сепаратизме противнико...,0,Human,Мэр Николаева Александр Сенкевич обвинил депут...,мэр николаева александр сенкевич обвинил депут...,125.300000,5.480000,1262,167,14
3,Цена нефти Brent поднялась выше 49 долларов за...,0,Human,Цены на нефть Brent выросли до 49 долларов за ...,цены на нефть brent выросли до 49 долларов за ...,119.100000,4.880383,1200,181,10
4,Журналист добавил в кофе майонез и вызвал омер...,0,Human,Американский спортивный журналист Джим Сэйлсбе...,американский спортивный журналист джим сэйлсбе...,133.333333,5.353846,1209,164,9
...,...,...,...,...,...,...,...,...,...,...
32763,В суд передали дело еще четырех фигурантов «бо...,1,RuGPT3_small,Новостная статья: В суд передали дело еще четы...,в суд передано дело ещё четырех обвиняемых по ...,172.166667,4.860000,3140,352,37
32764,Бразильский футболист попросил у Кадырова росс...,1,RuGPT3_small,Новостная статья: Бразильский футболист попрос...,и на этот раз не отказался от него в россии в ...,56.937500,2.615142,1858,174,61
32765,«ВКонтакте» выкупила у Дурова дата-центр,1,RuGPT3_small,Новостная статья: «ВКонтакте» выкупила у Дуров...,в лыткарино в состоянии банкротства вконтакте ...,123.562500,2.657534,2002,182,36
32766,Захарова доверилась любви в деле восстановлени...,1,RuGPT3_small,Новостная статья: Захарова доверилась любви в ...,оригинал взят у в захаровой доверились любви в...,80.636364,2.339568,1804,162,54


In [5]:
import re

def remove_news(text):
    return text.replace('Новостная статья:', '').replace('*Примечание: В данной статье приведены гипотетические данные и информация.*', '')

def remove_between_stars(text):
    return re.sub(r'\*\*(.*?)\*\*', '', text)

def remove_title(row):
    title = row['title']
    text = row['text']
    index = text.find(title)
    if index != -1:
        return remove_news(text[:index] + text[index + len(title):])
    else:
        return remove_news(remove_between_stars(text))
    

df['text_without_title'] = df.apply(remove_title, axis=1)

df


,title,generated,model,text,clean_text,avg_sentence_len,avg_words_len,char_count,word_count,sentence_count,text_without_title
0,Канье Уэст получил пожизненную подписку на Por...,0,Human,Популярный исполнитель Канье Уэст рассказал в ...,популярный исполнитель канье уэст рассказал в ...,87.384615,5.253968,1149,156,14,Популярный исполнитель Канье Уэст рассказал в ...
1,Сторонники Саакашвили сломали дверь микроавтоб...,0,Human,Сторонники Михаила Саакашвили взломали дверь м...,сторонники михаила саакашвили взломали дверь м...,103.000000,5.430000,1248,162,12,Сторонники Михаила Саакашвили взломали дверь м...
2,Мэр Николаева обвинил в сепаратизме противнико...,0,Human,Мэр Николаева Александр Сенкевич обвинил депут...,мэр николаева александр сенкевич обвинил депут...,125.300000,5.480000,1262,167,14,Мэр Николаева Александр Сенкевич обвинил депут...
3,Цена нефти Brent поднялась выше 49 долларов за...,0,Human,Цены на нефть Brent выросли до 49 долларов за ...,цены на нефть brent выросли до 49 долларов за ...,119.100000,4.880383,1200,181,10,Цены на нефть Brent выросли до 49 долларов за ...
4,Журналист добавил в кофе майонез и вызвал омер...,0,Human,Американский спортивный журналист Джим Сэйлсбе...,американский спортивный журналист джим сэйлсбе...,133.333333,5.353846,1209,164,9,Американский спортивный журналист Джим Сэйлсбе...
...,...,...,...,...,...,...,...,...,...,...,...
32763,В суд передали дело еще четырех фигурантов «бо...,1,RuGPT3_small,Новостная статья: В суд передали дело еще четы...,в суд передано дело ещё четырех обвиняемых по ...,172.166667,4.860000,3140,352,37,\nВ суд передано дело ещё четырех обвиняемых ...
32764,Бразильский футболист попросил у Кадырова росс...,1,RuGPT3_small,Новостная статья: Бразильский футболист попрос...,и на этот раз не отказался от него в россии в ...,56.937500,2.615142,1858,174,61,и на этот раз не отказался от него\n\nВ Росс...
32765,«ВКонтакте» выкупила у Дурова дата-центр,1,RuGPT3_small,Новостная статья: «ВКонтакте» выкупила у Дуров...,в лыткарино в состоянии банкротства вконтакте ...,123.562500,2.657534,2002,182,36,в Лыткарино»\n\nВ состоянии банкротства «Вко...
32766,Захарова доверилась любви в деле восстановлени...,1,RuGPT3_small,Новостная статья: Захарова доверилась любви в ...,оригинал взят у в захаровой доверились любви в...,80.636364,2.339568,1804,162,54,\n\nОригинал взят у в Захаровой доверились л...


In [6]:
def remove_html_entities(text):
    entity_pattern = re.compile(r'&[a-zA-Z]+;')
    cleaned_text = re.sub(entity_pattern, ' ', text)
    return cleaned_text


df['text_without_title'] = df['text_without_title'].str.replace('\n', ' ')
df['text_without_title'] = df['text_without_title'].str.replace('\t', ' ')
df['text_without_title'] = df['text_without_title'].apply(remove_html_entities)
df['text_without_title'] = df['text_without_title'].str.replace('\xa0+', ' ')
df['text_without_title'] = df['text_without_title'].str.replace(r'\s{2,}', ' ', regex=True)
df['text_without_title'] = df['text_without_title'].str.replace('\*+', ' ')

In [7]:
df

,title,generated,model,text,clean_text,avg_sentence_len,avg_words_len,char_count,word_count,sentence_count,text_without_title
0,Канье Уэст получил пожизненную подписку на Por...,0,Human,Популярный исполнитель Канье Уэст рассказал в ...,популярный исполнитель канье уэст рассказал в ...,87.384615,5.253968,1149,156,14,Популярный исполнитель Канье Уэст рассказал в ...
1,Сторонники Саакашвили сломали дверь микроавтоб...,0,Human,Сторонники Михаила Саакашвили взломали дверь м...,сторонники михаила саакашвили взломали дверь м...,103.000000,5.430000,1248,162,12,Сторонники Михаила Саакашвили взломали дверь м...
2,Мэр Николаева обвинил в сепаратизме противнико...,0,Human,Мэр Николаева Александр Сенкевич обвинил депут...,мэр николаева александр сенкевич обвинил депут...,125.300000,5.480000,1262,167,14,Мэр Николаева Александр Сенкевич обвинил депут...
3,Цена нефти Brent поднялась выше 49 долларов за...,0,Human,Цены на нефть Brent выросли до 49 долларов за ...,цены на нефть brent выросли до 49 долларов за ...,119.100000,4.880383,1200,181,10,Цены на нефть Brent выросли до 49 долларов за ...
4,Журналист добавил в кофе майонез и вызвал омер...,0,Human,Американский спортивный журналист Джим Сэйлсбе...,американский спортивный журналист джим сэйлсбе...,133.333333,5.353846,1209,164,9,Американский спортивный журналист Джим Сэйлсбе...
...,...,...,...,...,...,...,...,...,...,...,...
32763,В суд передали дело еще четырех фигурантов «бо...,1,RuGPT3_small,Новостная статья: В суд передали дело еще четы...,в суд передано дело ещё четырех обвиняемых по ...,172.166667,4.860000,3140,352,37,В суд передано дело ещё четырех обвиняемых по...
32764,Бразильский футболист попросил у Кадырова росс...,1,RuGPT3_small,Новостная статья: Бразильский футболист попрос...,и на этот раз не отказался от него в россии в ...,56.937500,2.615142,1858,174,61,и на этот раз не отказался от него В России в...
32765,«ВКонтакте» выкупила у Дурова дата-центр,1,RuGPT3_small,Новостная статья: «ВКонтакте» выкупила у Дуров...,в лыткарино в состоянии банкротства вконтакте ...,123.562500,2.657534,2002,182,36,в Лыткарино» В состоянии банкротства «Вконтак...
32766,Захарова доверилась любви в деле восстановлени...,1,RuGPT3_small,Новостная статья: Захарова доверилась любви в ...,оригинал взят у в захаровой доверились любви в...,80.636364,2.339568,1804,162,54,Оригинал взят у в Захаровой доверились любви ...


In [8]:
def n_gram_freq(text, n):
    return nltk.FreqDist(ngrams(text.split(), n))

generated_text = df.loc[27494]['clean_text']

human_text = df.loc[10]['clean_text']

generated_2grams = n_gram_freq(generated_text, 2)
human_2grams = n_gram_freq(human_text, 2)

for (gram, freq) in generated_2grams.most_common(10):
    print(f"{gram}: {freq} в сгенерированном тексте")

for (gram, freq) in human_2grams.most_common(10):
    print(f"{gram}: {freq} в написанном человеком тексте")


('боб', 'вайнштейн'): 4 в сгенерированном тексте
('на', 'ужин'): 3 в сгенерированном тексте
('в', 'сексуальных'): 2 в сгенерированном тексте
('сексуальных', 'домогательствах'): 2 в сгенерированном тексте
('ее', 'на'): 2 в сгенерированном тексте
('к', 'интимной'): 2 в сгенерированном тексте
('интимной', 'близости'): 2 в сгенерированном тексте
('утверждает', 'что'): 2 в сгенерированном тексте
('что', 'боб'): 2 в сгенерированном тексте
('брата', 'вайнштейна'): 1 в сгенерированном тексте
('европейских', 'раввинов'): 2 в написанном человеком тексте
('пинхас', 'гольдшмидт'): 2 в написанном человеком тексте
('европейского', 'союза'): 2 в написанном человеком тексте
('в', 'том'): 2 в написанном человеком тексте
('том', 'есть'): 2 в написанном человеком тексте
('в', 'европе'): 2 в написанном человеком тексте
('сказал', 'гольдшмидт'): 2 в написанном человеком тексте
('по', 'его'): 2 в написанном человеком тексте
('его', 'словам'): 2 в написанном человеком тексте
('главный', 'раввин'): 1 в написа

In [9]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [10]:
nltk.data.path.append("D:/Programs/nltk_data")
generated_text = df.loc[27494]['text']
human_text = df.loc[10]['text']

generated_sentences = sent_tokenize(generated_text)
human_sentences = sent_tokenize(human_text)

generated_words = word_tokenize(generated_text)
human_words = word_tokenize(human_text)

print("Средняя длина предложения:")
print(f"Сгенерированный текст: {sum(len(s) for s in generated_sentences) / len(generated_sentences)}")
print(f"Текст, написанный человеком: {sum(len(s) for s in human_sentences) / len(human_sentences)}")

print("Средняя длина слова:")
print(f"Сгенерированный текст: {sum(len(w) for w in generated_words) / len(generated_words)}")
print(f"Текст, написанный человеком: {sum(len(w) for w in human_words) / len(human_words)}")


Средняя длина предложения:
Сгенерированный текст: 103.23076923076923
Текст, написанный человеком: 167.77777777777777
Средняя длина слова:
Сгенерированный текст: 5.277272727272727
Текст, написанный человеком: 5.60337552742616


In [11]:
def deviations_mean_std(array):
    return np.abs(array - np.mean(array)) / np.std(array)

# Вычисление отклонений от среднего для частот всех 2-грамм в сгенерированном и написанном человеком текстах
generated_2gram_freqs = np.array([freq for freq in generated_2grams.values()])
human_2gram_freqs = np.array([freq for freq in human_2grams.values()])

# Выявление аномальных 2-грамм
generated_outliers = deviations_mean_std(generated_2gram_freqs)
human_outliers = deviations_mean_std(human_2gram_freqs)

# Печать наиболее аномальных 2-грамм
print("Наиболее аномальные 2-граммы в сгенерированном тексте:")
for (gram, outlier) in sorted(zip(generated_2grams.keys(), generated_outliers), key=lambda x: x[1], reverse=True)[:10]:
    print(f"{gram}: {outlier}")

print("Наиболее аномальные 2-граммы в написанном человеком тексте:")
for (gram, outlier) in sorted(zip(human_2grams.keys(), human_outliers), key=lambda x: x[1], reverse=True)[:10]:
    print(f"{gram}: {outlier}")


Наиболее аномальные 2-граммы в сгенерированном тексте:
('боб', 'вайнштейн'): 8.956384420396093
('на', 'ужин'): 5.902683827537234
('в', 'сексуальных'): 2.8489832346783763
('сексуальных', 'домогательствах'): 2.8489832346783763
('ее', 'на'): 2.8489832346783763
('к', 'интимной'): 2.8489832346783763
('интимной', 'близости'): 2.8489832346783763
('утверждает', 'что'): 2.8489832346783763
('что', 'боб'): 2.8489832346783763
('брата', 'вайнштейна'): 0.2047173581804819
Наиболее аномальные 2-граммы в написанном человеком тексте:
('европейских', 'раввинов'): 4.2946995755750415
('пинхас', 'гольдшмидт'): 4.2946995755750415
('европейского', 'союза'): 4.2946995755750415
('в', 'том'): 4.2946995755750415
('том', 'есть'): 4.2946995755750415
('в', 'европе'): 4.2946995755750415
('сказал', 'гольдшмидт'): 4.2946995755750415
('по', 'его'): 4.2946995755750415
('его', 'словам'): 4.2946995755750415
('главный', 'раввин'): 0.23284515771189962


In [12]:
df.loc[27492]['text']

'Холли Берри спродюсирует сериал про Ганнибала\n\nАктриса Холли Берри, известная по ролям в таких фильмах, как "Люди Икс" и "Джеймс Бонд", готовится к новому проекту - она станет продюсером сериала о Ганнибале Лектере.\n\nГаннибал Лектер - это персонаж из серии романов Томаса Харриса, который был впервые представлен в книге "Красный дракон". Этот персонаж стал одним из самых известных и любимых злодеев в истории кино.\n\nСериал будет основан на романах Харриса и расскажет историю Ганнибала Лектера до того, как он стал главным героем трилогии "Молчание ягнят". В сериале будут показаны его ранние годы, когда он еще не был известен как убийца и каннибал.\n\nХолли Берри, которая уже имеет опыт работы в качестве продюсера, заявила, что она очень рада возможности воплотить в жизнь этот проект. Она считает, что сериал о Ганнибале Лектере может стать настоящим шедевром и привлечь внимание зрителей со всего мира.\n\nСъемки сериала начнутся в ближайшее время, а премьера ожидается в 2023 году.'

In [13]:
df.loc[12]['clean_text']

'в туркмении учредили медаль в честь отца действующего президента республики гурбангулы бердымухамедова  мяликгулы бердымухамедова об этом сообщает хроника туркменистана за соответствующее решение единогласно проголосовал меджлис  однопалатный парламент страны как отмечает ресурс 12 августа глава государства представил общественности свою новую книгу посвященную отцу и его работе в органах внутренних дел в декабре 2016го сообщалось что в средних школах страны потребовали использовать цитаты из книги бердымухамедова источник мудрости в программу туркменской литературы в учебных заведениях также включены несколько книг президента в частности произведения живая легенда имя доброе нетленно птица государства и предвестники народного счастья ранее в учебных заведениях республики изучали трактат рухнама авторства первого президента сапармурата ниязова туркменбаши содержащий свод нравственных правил и рассуждения о туркменском народе после смерти ниязова в 2006 году и прихода к власти бердымух

In [14]:
generated_text = df.loc[27492]['text']
human_text = df.loc[12]['text']

flesch_kincaid_index_generated = textstat.flesch_kincaid_grade(generated_text)
flesch_kincaid_index_human = textstat.flesch_kincaid_grade(human_text)

print("Индекс удобочитаемости Флеша-Кинкейда generated:", flesch_kincaid_index_generated)
print("Индекс удобочитаемости Флеша-Кинкейда human:", flesch_kincaid_index_human)

Индекс удобочитаемости Флеша-Кинкейда generated: 3.6
Индекс удобочитаемости Флеша-Кинкейда human: 2.6


In [15]:
import textstat

# Текст для анализа
text = 'Матерный твит Пэрис Джексон о смерти связали со стрельбой в техасской церкви Вчера, 17 июня, в городе Сан-Антонио, штат Техас, произошла трагедия - в местной церкви произошла стрельба, в результате которой погибло несколько человек. Это событие вызвало широкий резонанс в обществе и привлекло внимание многих знаменитостей.\n\nОднако, среди всех звезд, которые выразили свои соболезнования и поддержку пострадавшим, особое внимание привлекла Пэрис Джексон. Дочь Майкла Джексона, известная своими неоднозначными высказываниями в социальных сетях, написала в своем Twitter матерный твит, который был связан со стрельбой в церкви.\n\n"Я не могу поверить, что это произошло снова. Я не могу поверить, что это произошло в церкви. Я не могу поверить, что это произошло в Техасе. Я не могу поверить, что это произошло в Америке. Я не могу поверить, что это произошло в мире. Я не могу поверить, что это произошло в моей жизни", - написала Пэрис Джексон.\n\nЭтот твит вызвал бурное обсуждение в социальных сетях. Многие пользователи выразили свое возмущение и осуждение поведения Пэрис Джексон, считая, что такие слова недопустимы в такой трагический момент. Однако, есть и те, кто поддержал ее, считая, что она имеет право выражать свои эмоции так, как ей удобно.\n\nСтрельба в церкви Сан-Антонио стала одним из самых страшных событий в истории Техаса. По последним данным, погибло 26 человек, а еще 20 получили ранения. Полиция продолжает расследование этого преступления, чтобы установить мотивы и личность преступника.\n\nПэрис Джексон, несмотря на свою скандальную репутацию, выразила свою поддержку пострадавшим и выразила надежду на то, что такое больше никогда не повторится.'


# Вычисляем необходимые параметры
word_count = len(text.split())  # Количество слов
sentence_count = text.count('.') + text.count('!') + text.count('?')  # Количество предложений
syllable_count = textstat.syllable_count(text)  # Количество слогов

# Вычисляем индекс удобочитаемости Флеша-Кинкейда
flesch_kincaid_index = 0.39 * (word_count / sentence_count) + 11.8 * (syllable_count / word_count) - 15.59

print("Индекс удобочитаемости Флеша-Кинкейда:", flesch_kincaid_index)


Индекс удобочитаемости Флеша-Кинкейда: 1.805561932089912


In [16]:
import nltk
from nltk.lm import MLE
from nltk.lm.preprocessing import padded_everygram_pipeline

# Пример текста
generated_text = df.loc[27492]['text']
human_text = df.loc[12]['text']
# Разделение текста на предложения и токенизация
sentences = nltk.sent_tokenize(generated_text)
tokenized_text = [nltk.word_tokenize(sentence.lower()) for sentence in sentences]

# Подготовка данных для построения модели
n = 2  # порядок n-граммы
train_data, padded_text = padded_everygram_pipeline(n, tokenized_text)

# Обучение модели
model = MLE(n)  # Максимальное правдоподобие оценок (MLE)
model.fit(train_data, padded_text)

# Рассчитываем перплексию
perplexity = model.perplexity(tokenized_text)
print("Perplexity:", perplexity)


Perplexity: inf


In [17]:
from scipy.stats import zscore
import numpy as np

# Пример текста
generated_text = df.loc[27492]['clean_text']
human_text = df.loc[12]['clean_text']
# Разделение текста на слова и подсчет их встречаемости
word_counts = nltk.FreqDist(nltk.word_tokenize(generated_text.lower()))

# Преобразование частот встречаемости в массив
counts_array = np.array(list(word_counts.values()))

# Рассчитываем Z-оценку
z_scores = zscore(counts_array)

# Вычисляем всплески
burstiness = sum(z_scores > 1.96)  # Значение Z-оценки для p = 0.05
print("Burstiness:", burstiness)


Burstiness: 4


In [35]:
titles = df.groupby('title').filter(lambda x: len(x) >= 2).title.unique()[:200]
len(titles)

200

In [57]:
df[(df.title.isin(titles[:200])) & (df.generated == 1)].sort_values(by=['title']).head(200)

,title,generated,model,text,clean_text,avg_sentence_len,avg_words_len,char_count,word_count,sentence_count,text_without_title
20828,11-летний мальчик сдал самый сложный тест по м...,1,YandexGPT lite,11-летний школьник побил рекорд по самому сло...,11летний школьник побил рекорд по самому сложн...,100.076923,4.969298,1317,178,12,11-летний школьник побил рекорд по самому сло...
31572,12-летнюю девочку выгнали из школы за голубые ...,1,RuGPT3_small,Новостная статья: 12-летнюю девочку выгнали из...,за 10 лет до своей смерти я познакомился с той...,79.187500,3.214076,1282,186,19,За 10 лет до своей смерти я познакомился с то...
21560,Chanel изменила свой классический флакон,1,YandexGPT lite,Chanel изменила дизайн своего классического ф...,chanel изменила дизайн своего классического фл...,78.000000,5.382609,717,90,9,Chanel изменила дизайн своего классического ф...
19820,Dolce & Gabbana поглумился над женщиной и разъ...,1,YandexGPT,**Dolce & Gabbana поглумился над женщиной и ра...,итальянский дом моды dolce gabbana оказался в...,98.222222,5.340278,896,123,9,Итальянский дом моды Dolce & Gabbana оказался...
18302,Google выбрал своего «Человека года»,1,YandexGPT,**Google выбрал своего «Человека года»**\n\nКо...,компания google официально объявила победителя...,110.000000,5.502857,1112,146,10,**** Компания Google официально объявила побед...
...,...,...,...,...,...,...,...,...,...,...,...
27087,Художница вживила себе три десятка чипов в раз...,1,GigaChat,Художница вживила себе три десятка чипов в раз...,художница вживила себе три десятка чипов в раз...,107.600000,4.737113,1089,166,10,Художница вживила себе три десятка чипов в раз...
31237,Цены на коньяк попросили поднять на треть,1,RuGPT3_small,Новостная статья: Цены на коньяк попросили под...,и сутки 30818928 nasedkin 20111122 020900 сбер...,190.166667,2.795640,1146,116,8,и сутки 30818928 nasedkin 2011-11-22 02:09:00...
18127,Четыре ребенка погибли при пожаре в Туле,1,YandexGPT,**Четыре ребёнка погибли при пожаре в Туле** \...,сегодня в туле произошла трагедия в результат...,68.000000,5.208791,556,76,8,Сегодня в Туле произошла трагедия — в результ...
26632,Чулпан Хаматова поведала о слезах в железных т...,1,GigaChat,"Чулпан Хаматова, известная российская актриса,...",чулпан хаматова известная российская актриса п...,113.923077,4.938462,1497,218,13,"Чулпан Хаматова, известная российская актриса,..."


In [52]:
df[(df.generated == 0) & (df.title.isin(titles[:200]))].sort_values(by=['title']).head(200)

,title,generated,model,text,clean_text,avg_sentence_len,avg_words_len,char_count,word_count,sentence_count,text_without_title
9173,11-летний мальчик сдал самый сложный тест по м...,0,Human,В Японии 11-летний школьник сдал тест по матем...,в японии 11летний школьник сдал тест по матема...,94.785714,5.351852,1340,185,14,В Японии 11-летний школьник сдал тест по матем...
9088,12-летнюю девочку выгнали из школы за голубые ...,0,Human,12-летнюю девочку выгнали из школы города Берд...,12летнюю девочку выгнали из школы города бердс...,95.454545,5.313953,1060,147,11,12-летнюю девочку выгнали из школы города Берд...
9176,Chanel изменила свой классический флакон,0,Human,Впервые в своей истории компания Chanel измени...,впервые в своей истории компания chanel измени...,135.777778,5.526042,1231,170,10,Впервые в своей истории компания Chanel измени...
9164,Dolce & Gabbana поглумился над женщиной и разъ...,0,Human,Итальянский модный дом Dolce & Gabbana выпусти...,итальянский модный дом dolce gabbana выпустил...,94.545455,5.195402,1050,147,11,Итальянский модный дом Dolce & Gabbana выпусти...
9014,Google выбрал своего «Человека года»,0,Human,Самой популярной персоной среди пользователей ...,самой популярной персоной среди пользователей ...,112.076923,5.168033,1469,209,13,Самой популярной персоной среди пользователей ...
...,...,...,...,...,...,...,...,...,...,...,...
9066,Чулпан Хаматова поведала о слезах в железных т...,0,Human,Народная артистка России Чулпан Хаматова вспом...,народная артистка россии чулпан хаматова вспом...,66.920000,4.709677,1696,243,26,Народная артистка России Чулпан Хаматова вспом...
9009,Шипулин разозлился и заговорил об «охоте на ве...,0,Human,"Российский биатлонист Антон Шипулин, который п...",российский биатлонист антон шипулин который по...,106.666667,5.463415,1292,172,12,"Российский биатлонист Антон Шипулин, который п..."
9181,Эмма Стоун пожелала впервые сняться обнаженной...,0,Human,"Актриса Эмма Стоун настояла на том, чтобы снят...",актриса эмма стоун настояла на том чтобы снять...,91.083333,4.660099,1104,161,12,"Актриса Эмма Стоун настояла на том, чтобы снят..."
9065,Эпичная дуэль незнакомцев на «факах» посреди у...,0,Human,Житель Нью-Йорка опубликовал в Twitter видео н...,житель ньюйорка опубликовал в twitter видео не...,95.916667,4.970149,1164,164,12,Житель Нью-Йорка опубликовал в Twitter видео н...


In [58]:
df_generated

,title,generated,model,text,clean_text,avg_sentence_len,avg_words_len,char_count,word_count,sentence_count,text_without_title
26584,Нанятая авторами «Игры престолов» координатор ...,1,GigaChat,"""Нанятая авторами «Игры престолов» координатор...",нанятая авторами игры престолов координатор по...,122.076923,5.000000,1603,232,13,"""Нанятая авторами «Игры престолов» координатор..."
17973,Группа Uriah Heep приедет в Россию,1,YandexGPT,### **Группа Uriah Heep приедет в Россию**\n\n...,легендарная группа uriah heep посетит россию с...,98.555556,5.317241,898,124,9,### Легендарная группа Uriah Heep посетит Росс...
19389,Бузова подарила победительнице конкурса подерж...,1,YandexGPT,**Бузова подарила победительнице конкурса поде...,популярная певица и телеведущая ольга бузова в...,93.083333,5.626437,1133,149,12,Популярная певица и телеведущая Ольга Бузова ...
25305,Выявлена новая польза кофе,1,YandexGPT lite,*Новая польза кофе: исследование подтверждает...,новая польза кофе исследование подтверждает по...,151.166667,5.914498,1832,233,12,*Новая польза кофе: исследование подтверждает...
18429,Белорусские милиционеры заставили подростка из...,1,YandexGPT,**Белорусские милиционеры заставили подростка ...,в белорусском городе гродно произошёл необычны...,75.272727,6.299145,842,101,11,**** В белорусском городе Гродно произошёл нео...
...,...,...,...,...,...,...,...,...,...,...,...
18560,Обвал рубля спасет российскую экономику,1,YandexGPT,**Обвал рубля спасёт российскую экономику?**\n...,в последнее время курс рубля резко упал по отн...,93.133333,5.495495,1413,189,14,В последнее время курс рубля резко упал по от...
17690,Гостью телешоу затравили за «одинокий» зуб во рту,1,YandexGPT,Гостью телешоу затравили за «одинокий» зуб во ...,гостью телешоу затравили за одинокий зуб во рт...,130.000000,5.571429,1313,178,10,Гостью телешоу затравили за «одинокий» зуб во ...
19262,Запах в новых автомобилях испортят ради китайцев,1,YandexGPT,**Запах в новых автомобилях испортят ради кита...,производители автомобилей готовы отказаться от...,127.800000,5.828125,1290,168,10,Производители автомобилей готовы отказаться о...
19809,Подмосковным бизнес-леди помогут добиться успеха,1,YandexGPT,**Подмосковным бизнес-леди помогут добиться ус...,в подмосковье стартовал новый проект направлен...,142.900000,5.980952,1442,182,10,"**** В Подмосковье стартовал новый проект, нап..."


In [60]:
df_generated = df[(df.title.isin(titles[:200])) & (df.generated == 1)].sort_values(by=['title']).head(200)
df_reference = df[(df.generated == 0) & (df.title.isin(titles[:200]))].sort_values(by=['title']).head(200)

In [62]:
df_reference

,title,generated,model,text,clean_text,avg_sentence_len,avg_words_len,char_count,word_count,sentence_count,text_without_title
9173,11-летний мальчик сдал самый сложный тест по м...,0,Human,В Японии 11-летний школьник сдал тест по матем...,в японии 11летний школьник сдал тест по матема...,94.785714,5.351852,1340,185,14,В Японии 11-летний школьник сдал тест по матем...
9088,12-летнюю девочку выгнали из школы за голубые ...,0,Human,12-летнюю девочку выгнали из школы города Берд...,12летнюю девочку выгнали из школы города бердс...,95.454545,5.313953,1060,147,11,12-летнюю девочку выгнали из школы города Берд...
9176,Chanel изменила свой классический флакон,0,Human,Впервые в своей истории компания Chanel измени...,впервые в своей истории компания chanel измени...,135.777778,5.526042,1231,170,10,Впервые в своей истории компания Chanel измени...
9164,Dolce & Gabbana поглумился над женщиной и разъ...,0,Human,Итальянский модный дом Dolce & Gabbana выпусти...,итальянский модный дом dolce gabbana выпустил...,94.545455,5.195402,1050,147,11,Итальянский модный дом Dolce & Gabbana выпусти...
9014,Google выбрал своего «Человека года»,0,Human,Самой популярной персоной среди пользователей ...,самой популярной персоной среди пользователей ...,112.076923,5.168033,1469,209,13,Самой популярной персоной среди пользователей ...
...,...,...,...,...,...,...,...,...,...,...,...
9066,Чулпан Хаматова поведала о слезах в железных т...,0,Human,Народная артистка России Чулпан Хаматова вспом...,народная артистка россии чулпан хаматова вспом...,66.920000,4.709677,1696,243,26,Народная артистка России Чулпан Хаматова вспом...
9009,Шипулин разозлился и заговорил об «охоте на ве...,0,Human,"Российский биатлонист Антон Шипулин, который п...",российский биатлонист антон шипулин который по...,106.666667,5.463415,1292,172,12,"Российский биатлонист Антон Шипулин, который п..."
9181,Эмма Стоун пожелала впервые сняться обнаженной...,0,Human,"Актриса Эмма Стоун настояла на том, чтобы снят...",актриса эмма стоун настояла на том чтобы снять...,91.083333,4.660099,1104,161,12,"Актриса Эмма Стоун настояла на том, чтобы снят..."
9065,Эпичная дуэль незнакомцев на «факах» посреди у...,0,Human,Житель Нью-Йорка опубликовал в Twitter видео н...,житель ньюйорка опубликовал в twitter видео не...,95.916667,4.970149,1164,164,12,Житель Нью-Йорка опубликовал в Twitter видео н...


In [63]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu


nltk.download('punkt')

def calculate_bleu_scores(references, candidates):
    
    individual_scores = [sentence_bleu(ref, cand) for ref, cand in zip(references, candidates)]
    corpus_score = corpus_bleu(references, candidates)
    return individual_scores, corpus_score


df_generated = df[(df.title.isin(titles[:200])) & (df.generated == 1)].sort_values(by=['title']).head(200)
df_reference = df[(df.generated == 0) & (df.title.isin(titles[:200]))].sort_values(by=['title']).head(200)

tokenized_references = [[nltk.word_tokenize(sent, language='russian') for sent in refs] for refs in df_reference['text_without_title']]
tokenized_candidates = [nltk.word_tokenize(sent, language='russian') for sent in df_generated['text_without_title']]

individual_scores, corpus_score = calculate_bleu_scores(tokenized_references, tokenized_candidates)

print(f"Individual BLEU scores: {individual_scores}")
print(f"Corpus BLEU score: {corpus_score}")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Alyonka\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
c:\Users\Alyonka\AppData\Local\Programs\Python\Python38\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\Users\Alyonka\AppData\Local\Programs\Python\Python38\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\Users\Alyonka\AppData\Local\Programs\Python\Python38\lib\site-packages\nltk\translate\bleu_score.py:552: 

Individual BLEU scores: [7.766026503126587e-232, 8.816720535482148e-232, 1.0003688322288243e-231, 9.80705635520132e-232, 8.764792329201055e-232, 1.1121803032788522e-231, 8.319100378795605e-232, 9.303268425030479e-232, 9.61304324412885e-232, 8.988710187084069e-232, 7.9677403708478e-232, 9.09498222341053e-232, 8.22037241180542e-232, 7.871671351844725e-232, 8.561894227089738e-232, 7.652397822812672e-232, 7.784451369270533e-232, 8.099315788452891e-232, 8.519704578791637e-232, 9.150285868297663e-232, 7.263783498533385e-232, 8.572367511776387e-232, 7.992219124248642e-232, 8.87866851697127e-232, 8.102834935834574e-232, 9.17251609423826e-232, 8.169352971200103e-232, 9.577802377303848e-232, 8.133957508410188e-232, 8.614911585158347e-232, 1.0678008316969705e-231, 8.614911585158347e-232, 9.594503055152632e-232, 9.594503055152632e-232, 9.145094214651507e-232, 1.2508498911928379e-231, 8.418450523686647e-232, 1.0080942617254966e-231, 8.871365973288784e-232, 8.42953628495304e-232, 8.749210414557588e-

In [50]:
df_merged = pd.read_csv("dataset_generated_to_not.csv", index_col='Unnamed: 0').reset_index(drop=True)
df_merged = df_merged.drop(df_merged[df_merged.text.isna()].index)
df_merged = df_merged.drop(df_merged[df_merged.generated_text_clean.isna()].index)

In [57]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24375 entries, 0 to 24376
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   title                         24375 non-null  object 
 1   generated_text                24375 non-null  object 
 2   generated                     24375 non-null  int64  
 3   model                         24375 non-null  object 
 4   generated_text_without_title  24375 non-null  object 
 5   generated_text_clean          24375 non-null  object 
 6   clean_text                    24375 non-null  object 
 7   text                          24375 non-null  object 
 8   rouge1_f1                     24375 non-null  float64
 9   rouge1_precision              24375 non-null  float64
 10  rouge1_recall                 24375 non-null  float64
 11  rouge2_f1                     24375 non-null  float64
 12  rouge2_precision              24375 non-null  float64
 13  rouge2

In [53]:
stemmer = SnowballStemmer("russian")

def preprocess_text(text):
    tokens = word_tokenize(text)
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)


scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=False)

def calculate_rouge_scores(row):
    generated_text = preprocess_text(row['generated_text_clean'])
    reference_text = preprocess_text(row['clean_text'])
    scores = scorer.score(reference_text, generated_text)
    
    rouge1_f1 = scores['rouge1'].fmeasure
    rouge1_precision = scores['rouge1'].precision
    rouge1_recall = scores['rouge1'].recall

    rouge2_f1 = scores['rouge2'].fmeasure
    rouge2_precision = scores['rouge2'].precision
    rouge2_recall = scores['rouge2'].recall

    rougeL_f1 = scores['rougeL'].fmeasure
    rougeL_precision = scores['rougeL'].precision
    rougeL_recall = scores['rougeL'].recall

    return pd.Series({
        'rouge1_f1': rouge1_f1,
        'rouge1_precision': rouge1_precision,
        'rouge1_recall': rouge1_recall,
        'rouge2_f1': rouge2_f1,
        'rouge2_precision': rouge2_precision,
        'rouge2_recall': rouge2_recall,
        'rougeL_f1': rougeL_f1,
        'rougeL_precision': rougeL_precision,
        'rougeL_recall': rougeL_recall,
    })

df_merged = df_merged.join(df_merged.apply(calculate_rouge_scores, axis=1))

df_merged

,title,generated_text,generated,model,generated_text_without_title,generated_text_clean,clean_text,text,rouge1_f1,rouge1_precision,rouge1_recall,rouge2_f1,rouge2_precision,rouge2_recall,rougeL_f1,rougeL_precision,rougeL_recall
0,В Испании арестован польский оружейный барон,**В Испании арестован польский оружейный барон...,1,YandexGPT,****\n\nПравоохранительные органы Испании арес...,правоохранительные органы испании арестовали г...,испанская полиция сообщила об аресте гражданин...,Испанская полиция сообщила об аресте гражданин...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
1,Пожар в жилом доме в Солнцево потушен,**Пожар в жилом доме в Солнцево потушен**\n\nС...,1,YandexGPT,"\n\nСегодня в жилом доме, расположенном в моск...",сегодня в жилом доме расположенном в московско...,пожар возникший в квартире жилого дома располо...,"Пожар, возникший в квартире жилого дома, распо...",0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
2,В Генпрокуратуре заявили о наличии обманутых д...,**В Генпрокуратуре заявили о наличии обманутых...,1,YandexGPT,\n\nГенеральная прокуратура России провела про...,генеральная прокуратура россии провела проверк...,проблемы с обманутыми дольщиками существуют в ...,Проблемы с обманутыми дольщиками существуют в ...,0.200000,1.000000,0.111111,0.0,0.0,0.0,0.200000,1.000000,0.111111
3,"Фонд ""Федерация"" отсудил у ""Коммерсанта"" три т...",**Фонд «Федерация» отсудил у «Коммерсанта» три...,1,YandexGPT,\n\nБлаготворительный фонд «Федерация» выиграл...,благотворительный фонд федерация выиграл дело ...,иск фонда федерация к газете коммерсантъ по по...,"Иск фонда ""Федерация"" к газете ""Коммерсантъ"" п...",0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
4,Российские банкиры считают невозможным снижени...,**Российские банкиры считают невозможным сниже...,1,YandexGPT,\n\nВ условиях экономической нестабильности и ...,в условиях экономической нестабильности и рост...,снижение ставок по ипотечным кредитам невозмож...,Снижение ставок по ипотечным кредитам невозмож...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24372,В Арктике нашли тропические леса,Новостная статья: В Арктике нашли тропические ...,1,RuGPT3_small pre-trained,", которые могут быть долгожителями. Ученые пр...",которые могут быть долгожителями ученые проана...,британские ученые в ходе экспедиции на норвежс...,Британские ученые в ходе экспедиции на норвежс...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
24373,Доллар упал ниже 70 рублей,Новостная статья: Доллар упал ниже 70 рублей в...,1,RuGPT3_small pre-trained,впервые с начала года. Об этом свидетельству...,впервые с начала года об этом свидетельствуют ...,курс доллара на московской бирже вечером в сре...,Курс доллара на Московской бирже вечером в сре...,0.103448,0.078947,0.150000,0.0,0.0,0.0,0.068966,0.052632,0.100000
24374,Подсчитаны доходы Роналду от Instagram,Новостная статья: Подсчитаны доходы Роналду от...,1,RuGPT3_small pre-trained,-аккаунта Роналдо. Роналдиньо является самым ...,аккаунта роналдо роналдиньо является самым выс...,нападающий мадридского реала криштиану роналду...,Нападающий мадридского «Реала» Криштиану Ронал...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
24375,Ученые нашли у кофе способность продлевать жизнь,Новостная статья: Ученые нашли у кофе способно...,1,RuGPT3_small pre-trained,"бактерий, которые производят кофеин. Исследо...",бактерий которые производят кофеин исследовани...,дветри чашки кофе в день снижают риск смерти о...,Две-три чашки кофе в день снижают риск смерти ...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000


In [54]:
df_merged.groupby(by=['model'])[['rouge1_f1', 
        'rouge2_f1',
        'rougeL_f1',
        ]].agg(['min', 'max', 'mean', 'median'])

rouge1_f1                            rouge2_f1  \
                               min       max      mean median       min   
model                                                                     
GigaChat                       0.0  0.800000  0.105263    0.0       0.0   
RuGPT3_small                   0.0  0.428571  0.032366    0.0       0.0   
RuGPT3_small pre-trained       0.0  0.571429  0.053832    0.0       0.0   
YandexGPT                      0.0  1.000000  0.098952    0.0       0.0   
YandexGPT 3 lite               0.0  1.000000  0.092993    0.0       0.0   
YandexGPT lite                 0.0  1.000000  0.091149    0.0       0.0   

                                                    rougeL_f1            \
                               max      mean median       min       max   
model                                                                     
GigaChat                  0.666667  0.024299    0.0       0.0  0.800000   
RuGPT3_small              0.133333  0.001163    0.0       0.0  0.428571   
RuGPT3_small pre-trained  0.400000  0.004381    0.0       0.0  0.571429   
YandexGPT                 0.857143  0.025066    0.0       0.0  1.000000   
YandexGPT 3 lite          1.000000  0.023472    0.0       0.0  1.000000   
YandexGPT lite            1.000000  0.019802    0.0       0.0  1.000000   

                                           
                              mean median  
model                                      
GigaChat                  0.100819    0.0  
RuGPT3_small              0.028788    0.0  
RuGPT3_small pre-trained  0.049288    0.0  
YandexGPT                 0.094353    0.0  
YandexGPT 3 lite          0.090188    0.0  
YandexGPT lite            0.086248    0.0

In [55]:
df_merged

,title,generated_text,generated,model,generated_text_without_title,generated_text_clean,clean_text,text,rouge1_f1,rouge1_precision,rouge1_recall,rouge2_f1,rouge2_precision,rouge2_recall,rougeL_f1,rougeL_precision,rougeL_recall
0,В Испании арестован польский оружейный барон,**В Испании арестован польский оружейный барон...,1,YandexGPT,****\n\nПравоохранительные органы Испании арес...,правоохранительные органы испании арестовали г...,испанская полиция сообщила об аресте гражданин...,Испанская полиция сообщила об аресте гражданин...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
1,Пожар в жилом доме в Солнцево потушен,**Пожар в жилом доме в Солнцево потушен**\n\nС...,1,YandexGPT,"\n\nСегодня в жилом доме, расположенном в моск...",сегодня в жилом доме расположенном в московско...,пожар возникший в квартире жилого дома располо...,"Пожар, возникший в квартире жилого дома, распо...",0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
2,В Генпрокуратуре заявили о наличии обманутых д...,**В Генпрокуратуре заявили о наличии обманутых...,1,YandexGPT,\n\nГенеральная прокуратура России провела про...,генеральная прокуратура россии провела проверк...,проблемы с обманутыми дольщиками существуют в ...,Проблемы с обманутыми дольщиками существуют в ...,0.200000,1.000000,0.111111,0.0,0.0,0.0,0.200000,1.000000,0.111111
3,"Фонд ""Федерация"" отсудил у ""Коммерсанта"" три т...",**Фонд «Федерация» отсудил у «Коммерсанта» три...,1,YandexGPT,\n\nБлаготворительный фонд «Федерация» выиграл...,благотворительный фонд федерация выиграл дело ...,иск фонда федерация к газете коммерсантъ по по...,"Иск фонда ""Федерация"" к газете ""Коммерсантъ"" п...",0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
4,Российские банкиры считают невозможным снижени...,**Российские банкиры считают невозможным сниже...,1,YandexGPT,\n\nВ условиях экономической нестабильности и ...,в условиях экономической нестабильности и рост...,снижение ставок по ипотечным кредитам невозмож...,Снижение ставок по ипотечным кредитам невозмож...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24372,В Арктике нашли тропические леса,Новостная статья: В Арктике нашли тропические ...,1,RuGPT3_small pre-trained,", которые могут быть долгожителями. Ученые пр...",которые могут быть долгожителями ученые проана...,британские ученые в ходе экспедиции на норвежс...,Британские ученые в ходе экспедиции на норвежс...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
24373,Доллар упал ниже 70 рублей,Новостная статья: Доллар упал ниже 70 рублей в...,1,RuGPT3_small pre-trained,впервые с начала года. Об этом свидетельству...,впервые с начала года об этом свидетельствуют ...,курс доллара на московской бирже вечером в сре...,Курс доллара на Московской бирже вечером в сре...,0.103448,0.078947,0.150000,0.0,0.0,0.0,0.068966,0.052632,0.100000
24374,Подсчитаны доходы Роналду от Instagram,Новостная статья: Подсчитаны доходы Роналду от...,1,RuGPT3_small pre-trained,-аккаунта Роналдо. Роналдиньо является самым ...,аккаунта роналдо роналдиньо является самым выс...,нападающий мадридского реала криштиану роналду...,Нападающий мадридского «Реала» Криштиану Ронал...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
24375,Ученые нашли у кофе способность продлевать жизнь,Новостная статья: Ученые нашли у кофе способно...,1,RuGPT3_small pre-trained,"бактерий, которые производят кофеин. Исследо...",бактерий которые производят кофеин исследовани...,дветри чашки кофе в день снижают риск смерти о...,Две-три чашки кофе в день снижают риск смерти ...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000


In [56]:
df_merged.to_csv('df_merged_rouge.csv')

In [35]:
similarity_dataset = pd.read_csv('merged_df_similarity_metrics.csv')

In [37]:
similarity_dataset.groupby(by=['model'])[['rouge1_f1',  'sts_score',
        'rouge2_f1',
        'rougeL_f1',
         'cosine_similarity_calculation_']].agg(['min', 'max', 'mean', 'median'])

rouge1_f1                            sts_score  \
                               min       max      mean median       min   
model                                                                     
GigaChat                       0.0  0.750000  0.121746    0.0  0.276286   
RuGPT3_small                   0.0  0.250000  0.026834    0.0 -0.026013   
RuGPT3_small pre-trained       0.0  0.307692  0.057546    0.0  0.151682   
YandexGPT                      0.0  1.000000  0.095219    0.0  0.269018   
YandexGPT lite                 0.0  0.800000  0.088338    0.0  0.174362   

                                                       rouge2_f1            \
                               max      mean    median       min       max   
model                                                                        
GigaChat                  0.939538  0.739555  0.753822       0.0  0.608696   
RuGPT3_small              0.864772  0.485713  0.501844       0.0  0.080000   
RuGPT3_small pre-trained  0.751413  0.514435  0.544563       0.0  0.129032   
YandexGPT                 0.945594  0.757109  0.771712       0.0  0.750000   
YandexGPT lite            0.955731  0.748739  0.764336       0.0  0.608696   

                                          rougeL_f1                      \
                              mean median       min       max      mean   
model                                                                     
GigaChat                  0.030217    0.0       0.0  0.750000  0.117228   
RuGPT3_small              0.000356    0.0       0.0  0.250000  0.023295   
RuGPT3_small pre-trained  0.003795    0.0       0.0  0.303030  0.049841   
YandexGPT                 0.023353    0.0       0.0  1.000000  0.091291   
YandexGPT lite            0.019337    0.0       0.0  0.727273  0.083594   

                                cosine_similarity_calculation_            \
                         median                            min       max   
model                                                                      
GigaChat                    0.0                      -0.610862  0.298853   
RuGPT3_small                0.0                      -0.594343  0.464182   
RuGPT3_small pre-trained    0.0                      -0.512048  0.973510   
YandexGPT                   0.0                      -0.620092  0.862111   
YandexGPT lite              0.0                      -0.618920  0.951010   

                                              
                              mean    median  
model                                         
GigaChat                 -0.550868 -0.562172  
RuGPT3_small             -0.519942 -0.551959  
RuGPT3_small pre-trained  0.444286  0.637651  
YandexGPT                -0.547667 -0.572249  
YandexGPT lite           -0.502226 -0.556072

In [60]:
df_merged

,title,generated_text,generated,model,generated_text_without_title,generated_text_clean,clean_text,text,rouge1_f1,rouge1_precision,rouge1_recall,rouge2_f1,rouge2_precision,rouge2_recall,rougeL_f1,rougeL_precision,rougeL_recall
0,В Испании арестован польский оружейный барон,**В Испании арестован польский оружейный барон...,1,YandexGPT,****\n\nПравоохранительные органы Испании арес...,правоохранительные органы испании арестовали г...,испанская полиция сообщила об аресте гражданин...,Испанская полиция сообщила об аресте гражданин...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
1,Пожар в жилом доме в Солнцево потушен,**Пожар в жилом доме в Солнцево потушен**\n\nС...,1,YandexGPT,"\n\nСегодня в жилом доме, расположенном в моск...",сегодня в жилом доме расположенном в московско...,пожар возникший в квартире жилого дома располо...,"Пожар, возникший в квартире жилого дома, распо...",0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
2,В Генпрокуратуре заявили о наличии обманутых д...,**В Генпрокуратуре заявили о наличии обманутых...,1,YandexGPT,\n\nГенеральная прокуратура России провела про...,генеральная прокуратура россии провела проверк...,проблемы с обманутыми дольщиками существуют в ...,Проблемы с обманутыми дольщиками существуют в ...,0.200000,1.000000,0.111111,0.0,0.0,0.0,0.200000,1.000000,0.111111
3,"Фонд ""Федерация"" отсудил у ""Коммерсанта"" три т...",**Фонд «Федерация» отсудил у «Коммерсанта» три...,1,YandexGPT,\n\nБлаготворительный фонд «Федерация» выиграл...,благотворительный фонд федерация выиграл дело ...,иск фонда федерация к газете коммерсантъ по по...,"Иск фонда ""Федерация"" к газете ""Коммерсантъ"" п...",0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
4,Российские банкиры считают невозможным снижени...,**Российские банкиры считают невозможным сниже...,1,YandexGPT,\n\nВ условиях экономической нестабильности и ...,в условиях экономической нестабильности и рост...,снижение ставок по ипотечным кредитам невозмож...,Снижение ставок по ипотечным кредитам невозмож...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24372,В Арктике нашли тропические леса,Новостная статья: В Арктике нашли тропические ...,1,RuGPT3_small pre-trained,", которые могут быть долгожителями. Ученые пр...",которые могут быть долгожителями ученые проана...,британские ученые в ходе экспедиции на норвежс...,Британские ученые в ходе экспедиции на норвежс...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
24373,Доллар упал ниже 70 рублей,Новостная статья: Доллар упал ниже 70 рублей в...,1,RuGPT3_small pre-trained,впервые с начала года. Об этом свидетельству...,впервые с начала года об этом свидетельствуют ...,курс доллара на московской бирже вечером в сре...,Курс доллара на Московской бирже вечером в сре...,0.103448,0.078947,0.150000,0.0,0.0,0.0,0.068966,0.052632,0.100000
24374,Подсчитаны доходы Роналду от Instagram,Новостная статья: Подсчитаны доходы Роналду от...,1,RuGPT3_small pre-trained,-аккаунта Роналдо. Роналдиньо является самым ...,аккаунта роналдо роналдиньо является самым выс...,нападающий мадридского реала криштиану роналду...,Нападающий мадридского «Реала» Криштиану Ронал...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
24375,Ученые нашли у кофе способность продлевать жизнь,Новостная статья: Ученые нашли у кофе способно...,1,RuGPT3_small pre-trained,"бактерий, которые производят кофеин. Исследо...",бактерий которые производят кофеин исследовани...,дветри чашки кофе в день снижают риск смерти о...,Две-три чашки кофе в день снижают риск смерти ...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000


In [66]:
df_with_perplexity = pd.read_csv('df_with_perplexity (1).csv').drop_duplicates(subset=['title', 'model']).reset_index(drop=True)
df_with_perplexity

,title,generated_text,generated,model,generated_text_without_title,generated_text_clean,clean_text,text,rouge1_f1,rouge1_precision,rouge1_recall,rouge2_f1,rouge2_precision,rouge2_recall,rougeL_f1,rougeL_precision,rougeL_recall,sts_score,perplexity,perplexity_original_text
0,В Испании арестован польский оружейный барон,**В Испании арестован польский оружейный барон...,1,YandexGPT,****\n\nПравоохранительные органы Испании арес...,правоохранительные органы испании арестовали г...,испанская полиция сообщила об аресте гражданин...,Испанская полиция сообщила об аресте гражданин...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.887151,5.974008,11.997695
1,Пожар в жилом доме в Солнцево потушен,**Пожар в жилом доме в Солнцево потушен**\n\nС...,1,YandexGPT,"\n\nСегодня в жилом доме, расположенном в моск...",сегодня в жилом доме расположенном в московско...,пожар возникший в квартире жилого дома располо...,"Пожар, возникший в квартире жилого дома, распо...",0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.899150,5.574790,6.068287
2,В Генпрокуратуре заявили о наличии обманутых д...,**В Генпрокуратуре заявили о наличии обманутых...,1,YandexGPT,\n\nГенеральная прокуратура России провела про...,генеральная прокуратура россии провела проверк...,проблемы с обманутыми дольщиками существуют в ...,Проблемы с обманутыми дольщиками существуют в ...,0.200000,1.000000,0.111111,0.0,0.0,0.0,0.200000,1.000000,0.111111,0.783440,6.180483,9.458086
3,"Фонд ""Федерация"" отсудил у ""Коммерсанта"" три т...",**Фонд «Федерация» отсудил у «Коммерсанта» три...,1,YandexGPT,\n\nБлаготворительный фонд «Федерация» выиграл...,благотворительный фонд федерация выиграл дело ...,иск фонда федерация к газете коммерсантъ по по...,"Иск фонда ""Федерация"" к газете ""Коммерсантъ"" п...",0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.719282,9.139601,17.563833
4,Российские банкиры считают невозможным снижени...,**Российские банкиры считают невозможным сниже...,1,YandexGPT,\n\nВ условиях экономической нестабильности и ...,в условиях экономической нестабильности и рост...,снижение ставок по ипотечным кредитам невозмож...,Снижение ставок по ипотечным кредитам невозмож...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.815478,7.054786,8.867599
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24399,В Арктике нашли тропические леса,Новостная статья: В Арктике нашли тропические ...,1,RuGPT3_small pre-trained,", которые могут быть долгожителями. Ученые пр...",которые могут быть долгожителями ученые проана...,британские ученые в ходе экспедиции на норвежс...,Британские ученые в ходе экспедиции на норвежс...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.570741,18.511623,10.586896
24400,Доллар упал ниже 70 рублей,Новостная статья: Доллар упал ниже 70 рублей в...,1,RuGPT3_small pre-trained,впервые с начала года. Об этом свидетельству...,впервые с начала года об этом свидетельствуют ...,курс доллара на московской бирже вечером в сре...,Курс доллара на Московской бирже вечером в сре...,0.103448,0.078947,0.150000,0.0,0.0,0.0,0.068966,0.052632,0.100000,0.706034,13.230332,6.446386
24401,Подсчитаны доходы Роналду от Instagram,Новостная статья: Подсчитаны доходы Роналду от...,1,RuGPT3_small pre-trained,-аккаунта Роналдо. Роналдиньо является самым ...,аккаунта роналдо роналдиньо является самым выс...,нападающий мадридского реала криштиану роналду...,Нападающий мадридского «Реала» Криштиану Ронал...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.675326,21.669037,9.384760
24402,Ученые нашли у кофе способность продлевать жизнь,Новостная статья: Ученые нашли у кофе способно...,1,RuGPT3_small pre-trained,"бактерий, которые производят кофеин. Исследо...",бактерий которые производят кофеин исследовани...,дветри чашки кофе в день снижают риск смерти о...,Две-три чашки кофе в день снижают риск смерти ...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00000

In [89]:
df_with_perplexity = df_with_perplexity.drop(df_with_perplexity[df_with_perplexity['perplexity'] > 150].index)
df_with_perplexity = df_with_perplexity.drop(df_with_perplexity[~ df_with_perplexity.model.isin(['GigaChat','RuGPT3_small','YandexGPT lite', 'YandexGPT 3 lite', 'YandexGPT', 'RuGPT3_small pre-trained'])].index)

In [86]:
df_with_perplexity

,title,generated_text,generated,model,generated_text_without_title,generated_text_clean,clean_text,text,rouge1_f1,rouge1_precision,rouge1_recall,rouge2_f1,rouge2_precision,rouge2_recall,rougeL_f1,rougeL_precision,rougeL_recall,sts_score,perplexity,perplexity_original_text
0,В Испании арестован польский оружейный барон,**В Испании арестован польский оружейный барон...,1,YandexGPT,****\n\nПравоохранительные органы Испании арес...,правоохранительные органы испании арестовали г...,испанская полиция сообщила об аресте гражданин...,Испанская полиция сообщила об аресте гражданин...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.887151,5.974008,11.997695
1,Пожар в жилом доме в Солнцево потушен,**Пожар в жилом доме в Солнцево потушен**\n\nС...,1,YandexGPT,"\n\nСегодня в жилом доме, расположенном в моск...",сегодня в жилом доме расположенном в московско...,пожар возникший в квартире жилого дома располо...,"Пожар, возникший в квартире жилого дома, распо...",0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.899150,5.574790,6.068287
2,В Генпрокуратуре заявили о наличии обманутых д...,**В Генпрокуратуре заявили о наличии обманутых...,1,YandexGPT,\n\nГенеральная прокуратура России провела про...,генеральная прокуратура россии провела проверк...,проблемы с обманутыми дольщиками существуют в ...,Проблемы с обманутыми дольщиками существуют в ...,0.200000,1.000000,0.111111,0.0,0.0,0.0,0.200000,1.000000,0.111111,0.783440,6.180483,9.458086
3,"Фонд ""Федерация"" отсудил у ""Коммерсанта"" три т...",**Фонд «Федерация» отсудил у «Коммерсанта» три...,1,YandexGPT,\n\nБлаготворительный фонд «Федерация» выиграл...,благотворительный фонд федерация выиграл дело ...,иск фонда федерация к газете коммерсантъ по по...,"Иск фонда ""Федерация"" к газете ""Коммерсантъ"" п...",0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.719282,9.139601,17.563833
4,Российские банкиры считают невозможным снижени...,**Российские банкиры считают невозможным сниже...,1,YandexGPT,\n\nВ условиях экономической нестабильности и ...,в условиях экономической нестабильности и рост...,снижение ставок по ипотечным кредитам невозмож...,Снижение ставок по ипотечным кредитам невозмож...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.815478,7.054786,8.867599
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24399,В Арктике нашли тропические леса,Новостная статья: В Арктике нашли тропические ...,1,RuGPT3_small pre-trained,", которые могут быть долгожителями. Ученые пр...",которые могут быть долгожителями ученые проана...,британские ученые в ходе экспедиции на норвежс...,Британские ученые в ходе экспедиции на норвежс...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.570741,18.511623,10.586896
24400,Доллар упал ниже 70 рублей,Новостная статья: Доллар упал ниже 70 рублей в...,1,RuGPT3_small pre-trained,впервые с начала года. Об этом свидетельству...,впервые с начала года об этом свидетельствуют ...,курс доллара на московской бирже вечером в сре...,Курс доллара на Московской бирже вечером в сре...,0.103448,0.078947,0.150000,0.0,0.0,0.0,0.068966,0.052632,0.100000,0.706034,13.230332,6.446386
24401,Подсчитаны доходы Роналду от Instagram,Новостная статья: Подсчитаны доходы Роналду от...,1,RuGPT3_small pre-trained,-аккаунта Роналдо. Роналдиньо является самым ...,аккаунта роналдо роналдиньо является самым выс...,нападающий мадридского реала криштиану роналду...,Нападающий мадридского «Реала» Криштиану Ронал...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.675326,21.669037,9.384760
24402,Ученые нашли у кофе способность продлевать жизнь,Новостная статья: Ученые нашли у кофе способно...,1,RuGPT3_small pre-trained,"бактерий, которые производят кофеин. Исследо...",бактерий которые производят кофеин исследовани...,дветри чашки кофе в день снижают риск смерти о...,Две-три чашки кофе в день снижают риск смерти ...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00000

In [87]:
df_with_perplexity[~ df_with_perplexity.model.isin(['GigaChat','RuGPT3_small','YandexGPT lite', 'YandexGPT 3 lite', 'YandexGPT', 'RuGPT3_small pre-trained'])]

,title,generated_text,generated,model,generated_text_without_title,generated_text_clean,clean_text,text,rouge1_f1,rouge1_precision,rouge1_recall,rouge2_f1,rouge2_precision,rouge2_recall,rougeL_f1,rougeL_precision,rougeL_recall,sts_score,perplexity,perplexity_original_text


In [91]:
df_with_perplexity.groupby(by=['model'])[['sts_score']].agg(['min', 'max', 'mean', 'median'])

sts_score                              
                               min       max      mean    median
model                                                           
GigaChat                  0.276286  0.949357  0.743014  0.756152
RuGPT3_small             -0.100604  0.926263  0.514444  0.532904
RuGPT3_small pre-trained -0.052333  0.905642  0.535352  0.563443
YandexGPT                -0.193830  0.953546  0.753172  0.769002
YandexGPT 3 lite          0.278011  0.939900  0.729774  0.744395
YandexGPT lite            0.174363  0.956613  0.755328  0.771522

In [92]:
df_with_perplexity.groupby(by=['model'])[['rouge1_f1', 
        'rouge2_f1',
        'rougeL_f1',
        ]].agg(['min', 'max', 'mean', 'median'])

rouge1_f1                            rouge2_f1  \
                               min       max      mean median       min   
model                                                                     
GigaChat                       0.0  0.800000  0.105263    0.0       0.0   
RuGPT3_small                   0.0  0.428571  0.032416    0.0       0.0   
RuGPT3_small pre-trained       0.0  0.571429  0.053832    0.0       0.0   
YandexGPT                      0.0  1.000000  0.098952    0.0       0.0   
YandexGPT 3 lite               0.0  1.000000  0.092993    0.0       0.0   
YandexGPT lite                 0.0  1.000000  0.091115    0.0       0.0   

                                                    rougeL_f1            \
                               max      mean median       min       max   
model                                                                     
GigaChat                  0.666667  0.024299    0.0       0.0  0.800000   
RuGPT3_small              0.133333  0.001164    0.0       0.0  0.428571   
RuGPT3_small pre-trained  0.400000  0.004381    0.0       0.0  0.571429   
YandexGPT                 0.857143  0.025066    0.0       0.0  1.000000   
YandexGPT 3 lite          1.000000  0.023472    0.0       0.0  1.000000   
YandexGPT lite            1.000000  0.019815    0.0       0.0  1.000000   

                                           
                              mean median  
model                                      
GigaChat                  0.100819    0.0  
RuGPT3_small              0.028827    0.0  
RuGPT3_small pre-trained  0.049288    0.0  
YandexGPT                 0.094353    0.0  
YandexGPT 3 lite          0.090188    0.0  
YandexGPT lite            0.086210    0.0